# Pylimmon Limit Monitoring Demo

The pylimmon functions `check_limit_msid` and `check_state_msid` are used to check for telemetry violations given the MSID and time span. These functions each return a list of tuple objects, where each tuple corresponds to a single violation. Each tuple includes (in the following order):
 - Time values for each recorded violating datapoint
 - Telemetry values for each recorded violating datapoint
 - Active limit or expected state (because some MSIDs have multiple sets)
 - Active set number (corresponding to limit above)
 - Limit type (e.g. caution high or expected state)
 
Pylimmon can also be used to query TDB values and trending limits/expected states, however this demo will focus just on limit monitoring.

Pylimmon, in its current state, is intended to provide a simple interface to limits/expected states and to provide basic checking tools. As each subystem has their own unique requirements, only minimal post processing is built into this package. It is up to the user to account for naming differences between Ska and GRETA MSIDs (e.g. telescope thermal widerange MSIDs), to consolidate violation data (e.g. toggles), and otherwise digest this data into a presentable format. 

In [2]:
import sys
import numpy as np
import jinja2 as ja
import cPickle as pickle
import os
from IPython.display import HTML
from Chandra.Time import DateTime
from Ska.engarchive import fetch_eng as fetch

os.environ["SKA_DATA"] = "/proj/sot/ska/data"
home = os.path.expanduser("~")
sys.path.append(home + '/AXAFLIB/pylimmon/')
import pylimmon

%matplotlib inline

# Currently there are some warnings related to soon to be deprecated code, ignore these for now.
old_settings = np.seterr(all='ignore')


For example, lets look at the violations for the ACA housing MSID, AACH1T. As you can see below, there are a number of instances where this MSID exceeded its caution high limit. Each instance is grouped into a tuple which covers the start and stop time of each violation. Initially each violation lasted for a single update, however eventually, they lasted for longer before toggling again as this location cooled.

If warning high violations were present, they would also be included in this list.

In [3]:
v = pylimmon.check_limit_msid('AACH1T', '2015:006:00:00:00.000', '2015:008:00:00:00.000')
v

[(array([  5.36970334e+08]),
  array([ 83.18948364]),
  array([ 83.]),
  array([0], dtype=int8),
  'caution_high'),
 (array([  5.36970498e+08]),
  array([ 83.18948364]),
  array([ 83.]),
  array([0], dtype=int8),
  'caution_high'),
 (array([  5.36970596e+08]),
  array([ 83.18948364]),
  array([ 83.]),
  array([0], dtype=int8),
  'caution_high'),
 (array([  5.36970662e+08]),
  array([ 83.18948364]),
  array([ 83.]),
  array([0], dtype=int8),
  'caution_high'),
 (array([  5.36970727e+08,   5.36970760e+08,   5.36970793e+08, ...,
           5.37046430e+08,   5.37046462e+08,   5.37046495e+08]),
  array([ 83.18948364,  83.18948364,  83.18948364, ...,  83.18948364,
          83.18948364,  83.18948364]),
  array([ 83.]),
  array([0], dtype=int8),
  'caution_high'),
 (array([  5.37046561e+08]),
  array([ 83.18948364]),
  array([ 83.]),
  array([0], dtype=int8),
  'caution_high'),
 (array([  5.37046626e+08,   5.37046659e+08,   5.37046692e+08,
           5.37046725e+08,   5.37046758e+08,   5.3704

_________________
Below some functions are defined to handle MSID naming differences between Ska and GRETA, to process a list of MSIDs, and to post-process the limit violation data. 

In [4]:
def check_violations(thermdict, t1, t2):
    """Check a list of MSIDs for limit/expected state violations.

    :param thermdict: Dictionary of MSID information (MSID name, condition type, etc.)
    :param t1: String containing start date in HOSC format
    :param t2: String containgin stop date in HOSC format

    """
    t1 = DateTime(t1).date
    t2 = DateTime(t2).date

    allviolations = {}
    missingmsids = []
    checkedmsids = []
    for key in thermdict.keys():
        greta_msid = thermdict[key]['greta_msid']
        try:
            if thermdict[key]['type'] == 'limit':
                if "wide" in greta_msid.lower():
                    violations = handle_widerange_cases(key, t1, t2, greta_msid)
                    checkedmsids.append(key)
                else:
                    violations = pylimmon.check_limit_msid(key, t1, t2, greta_msid=greta_msid)
                    checkedmsids.append(key)
            elif thermdict[key]['type'] == 'expst':
                violations = pylimmon.check_state_msid(key, t1, t2, greta_msid=greta_msid)
                checkedmsids.append(key)

            if len(violations) > 0:
                allviolations[key] = process_violations(key, violations)

        except IndexError:
            print('{} not in DB'.format(key))
            missingmsids.append(key)

    return allviolations, missingmsids, checkedmsids


def handle_widerange_cases(key, t1, t2, greta_msid):
    """Handle special widerange MSIDs.

    :param key: Name of MSID as represented in Ska Engineering Archive
    :param t1: String containing start time in HOSC format
    :param t2: String containgin stop time in HOSC format
    :greta_msid: Name of MSID as represented in GRETA

    Note: Some MSID names differ between Ska and GRETA. Widerange MSIDs are one such case. For 
    example OOBTHR35 is used for this measurement in both Ska and GRETA before this MSID was
    switched to widerange read mode. Afterwards GRETA uses OOBTHR35_WIDE whereas Ska still uses
    OOBTHR35 for continuity.
    """
    if DateTime(t2).secs <= DateTime('2014:342:16:30:00').secs:
        violations = pylimmon.check_limit_msid(key, t1, t2, greta_msid=key)
    elif DateTime(t1).secs >= DateTime('2014:342:16:33:00').secs:
        violations = pylimmon.check_limit_msid(key, t1, t2, greta_msid=greta_msid)
    else:
        t2_a = np.min((DateTime(t2).secs, DateTime('2014:342:16:30:00').secs))
        violations = pylimmon.check_limit_msid(key, t1, t2_a, greta_msid=key)
        t1_b = np.min((DateTime(t2).secs, DateTime('2014:342:16:33:00').secs))
        violations_b = pylimmon.check_limit_msid(key, t1_b, t2, greta_msid=greta_msid)

        violations.extend(violations_b)

    return violations


def process_violations(msid, violations):
    """Add contextual information for any limit/expected state violations.

    :param msid: Current mnemonic
    :param violations: List of individual violations (list of tuples)

    """
    data = fetch.Msid(msid, violations[0][0][0], violations[0][0][-1], stat='5min')
    try:
        desc = data.tdb.technical_name
    except:
        desc = 'No Description in TDB'

    violation_dict = {}
    for v in violations:
        limtype = v[-1]
        if 'high' in limtype.lower():
            if limtype not in violation_dict.keys():
                violation_dict.update({limtype: {'starttime': v[0][0], 'stoptime': v[0][-1],
                                                 'num_excursions': 1, 'extrema': np.max(v[1]),
                                                 'limit': v[2][0], 'setid': v[3][0],
                                                 'duration': v[0][-1] - v[0][0]}})
            else:
                violation_dict[limtype]['extrema'] = np.max(
                    (np.max(v[1]), violation_dict[limtype]['extrema']))
                violation_dict[limtype]['starttime'] = np.min(
                    (v[0][0], violation_dict[limtype]['starttime']))
                violation_dict[limtype]['stoptime'] = np.max(
                    (v[0][0], violation_dict[limtype]['stoptime']))
                violation_dict[limtype]['num_excursions'] = violation_dict[
                    limtype]['num_excursions'] + 1
                violation_dict[limtype]['duration'] = violation_dict[
                    limtype]['duration'] + v[0][-1] - v[0][0]

        elif 'low' in limtype.lower():
            if limtype not in violation_dict.keys():
                violation_dict.update({limtype: {'starttime': v[0][0], 'stoptime': v[0][-1],
                                                 'num_excursions': 1, 'extrema': np.min(v[1]),
                                                 'limit': v[2][0], 'setid': v[3][0],
                                                 'duration': v[0][-1] - v[0][0]}})
            else:
                violation_dict[limtype]['extrema'] = np.min(
                    (np.min(v[1]), violation_dict[limtype]['extrema']))
                violation_dict[limtype]['starttime'] = np.min(
                    (v[0][0], violation_dict[limtype]['starttime']))
                violation_dict[limtype]['stoptime'] = np.max(
                    (v[0][0], violation_dict[limtype]['stoptime']))
                violation_dict[limtype]['num_excursions'] = violation_dict[
                    limtype]['num_excursions'] + 1
                violation_dict[limtype]['duration'] = violation_dict[
                    limtype]['duration'] + v[0][-1] - v[0][0]

        elif 'state' in limtype.lower():
            if limtype not in violation_dict.keys():
                violation_dict.update({limtype: {'starttime': v[0][0], 'stoptime': v[0][-1],
                                                 'num_excursions': 1, 'extrema': v[1][0],
                                                 'limit': v[2][0], 'setid': v[3][0],
                                                 'duration': v[0][-1] - v[0][0]}})
            else:
                violation_dict[limtype]['starttime'] = np.min(
                    (v[0][0], violation_dict[limtype]['starttime']))
                violation_dict[limtype]['stoptime'] = np.max(
                    (v[0][0], violation_dict[limtype]['stoptime']))
                violation_dict[limtype]['num_excursions'] = violation_dict[
                    limtype]['num_excursions'] + 1
                violation_dict[limtype]['duration'] = violation_dict[
                    limtype]['duration'] + v[0][-1] - v[0][0]

    for limittype in ['warning_low', 'caution_low', 'caution_high', 'warning_high', 'state']:
        if limittype in violation_dict.keys():
            violation_dict[limittype]['duration'] = violation_dict[limittype]['duration'] / 3600.
            violation_dict[limittype]['description'] = desc
            violation_dict[limittype]['startdate'] = DateTime(
                violation_dict[limittype]['starttime']).date
            violation_dict[limittype]['stopdate'] = DateTime(
                violation_dict[limittype]['stoptime']).date

    return violation_dict


_________________
Next, check all MSIDs relevant to the thermal subsystem

In [5]:
thermdict, missing, notinarchive = pickle.load(open(home +
    '/AXAFDATA/weekly_report_data/thermalmsiddata.pkl','r'))
t1 = DateTime('2015:007:00:00:00.000').date
t2 = DateTime('2015:007:05:00:00.000').date

allviolations, missingmsids, checkedmsids = check_violations(thermdict, t1, t2)

# 3shtren and 4csdhav are not decommed correctly in the CXC archive
if '3shtren' in allviolations.keys():
    _ = allviolations.pop('3shtren')

if '4csdhav' in allviolations.keys():
    _ = allviolations.pop('4csdhav')

hours = (DateTime(t2).secs - DateTime(t1).secs)/3600.
print('\nChecked {} MSIDs at full resolution over {} hour period,' + 
      '{} MSIDs left unchecked (missing)\n'.format(
        len(checkedmsids), hours, len(missingmsids)) )


eob2ts1s not in DB
eotb3t1c not in DB
eob1ts1s not in DB
eoeb2rl3 not in DB
4m28irbx not in DB
4m5irbx not in DB
eob1ts2s not in DB

Checked {} MSIDs at full resolution over {} hour period,983 MSIDs left unchecked (missing)



_________________
Now show the list of violations in a table, using a Jinja template defined in the `templates` directory. 

In [6]:
env = ja.Environment(loader=ja.FileSystemLoader('./templates'))
template = env.get_template('index.htm')
webpage = template.render(violations=allviolations)
HTML(webpage)